<img src="img/logo_demcompare.png" width="100" align="right">

# Demcompare: srtm scenario

This notebook presents how to exploit the metrics and methods implemented in demcompare for a srtm DEM from data_samples.
A srtm reference data is used as a reference and a manually blurred and shifted data is used as tested data.

# Imports and external functions

In [ ]:
import pyproj # pyproj as first import is necessary

In [ ]:
from snippets.utils_notebook import *

In [ ]:
from IPython.display import HTML, display
import tabulate

# DEMs preparation

In [ ]:
from demcompare.dem_tools import load_dem

In [ ]:
input_ref_config = {
            "path" : "data/srtm/srtm_ref.tif",
            "zunit" : "m",
            "nodata": -9999
        }
input_sec_config = {
            "path" : "data/srtm/srtm_blurred_and_shifted_res.tif",
            "zunit" : "m",
            "nodata" : -32768,
        }

Loading the DEMs

In [ ]:
input_ref = load_dem(
    path=input_ref_config["path"], 
    zunit=input_ref_config["zunit"],
)

input_sec = load_dem(
    path=input_sec_config["path"], 
    zunit=input_sec_config["zunit"], 
)

DEMs should be coregistred to be aligned horizontally (see dedicated notebook for details)

In [ ]:
from demcompare.coregistration import Coregistration

In [ ]:
cfg = {
    "coregistration": {
        "method_name": "nuth_kaab_internal", #one method available for now
        "number_of_iterations": 6,
        "estimated_initial_shift_x": 0,
        "estimated_initial_shift_y": 0,
    }
}

In [ ]:
coregistration_ = Coregistration(cfg["coregistration"])
transformation = coregistration_.compute_coregistration(input_sec, input_ref)
reproj_ref = coregistration_.reproj_ref
reproj_sec = coregistration_.reproj_sec

Computing the slope for both DEMs with `compute_dem_slope`. The slope will be used to compute metrics.

In [ ]:
from demcompare.dem_tools import compute_dem_slope

In [ ]:
reproj_ref = compute_dem_slope(reproj_ref)
reproj_sec = compute_dem_slope(reproj_sec)

Import `DemProcessing` needed in the notebook

In [ ]:
from demcompare.dem_processing import DemProcessing

# Comparing the two DEMs independently

In this section, the 2 DEMs are compared independently, with visualizations and quality measures.

Create a dictonnary with a `statistics` section, and the `global` classification layer inside.
We won't use other classifications layers here, but it is possible.
Some metrics are selected to be calculated and analyzed with some methods later.

In [ ]:
cfg = {
    "statistics": {
        "global": {
        },
        "metrics":["mean","std","median","nmad","percentil_90","pdf","cdf"]
    }
}

Import `StatsProcessing` in order to be able to produce statistics.

In [ ]:
from demcompare.stats_processing import StatsProcessing

Create a `stats_processing_` object for the two DEMs independently.

In [ ]:
stats_processing_ref = StatsProcessing(cfg['statistics'], reproj_ref)
stats_processing_sec = StatsProcessing(cfg['statistics'], reproj_sec)

## DEMs visualizations

Compute statistics for the two DEMs independently.
The statistics computed here correspond to additional visualizations:
- `hillshade`
- `svf` (sky-view factor)

In [ ]:
stats_dataset_ref = stats_processing_ref.compute_stats(metrics = ["hillshade", "svf"])
stats_dataset_sec = stats_processing_sec.compute_stats(metrics = ["hillshade", "svf"])

In [ ]:
hillshade_ref = stats_dataset_ref.get_classification_layer_metric(classification_layer = 'global', metric="hillshade", classif_class=0)
hillshade_sec = stats_dataset_sec.get_classification_layer_metric(classification_layer = 'global', metric="hillshade", classif_class=0)
svf_ref = stats_dataset_ref.get_classification_layer_metric(classification_layer = 'global', metric="svf", classif_class=0)
svf_sec = stats_dataset_sec.get_classification_layer_metric(classification_layer = 'global', metric="svf", classif_class=0)

Plot the three visualizations side by side, for reference DEM and the DEM to evaluate. It is possible to set the minimum and maximum values of the colorbar range for the different representations.  

In [ ]:
vmin_height, vmax_height = 1800, 2000
vmin_hs, vmax_hs = 38, 250
vmin_svf, vmax_svf = 0, 255
colorbar_range = [vmin_height, vmax_height,vmin_hs, vmax_hs,vmin_svf, vmax_svf]

plot_visualizations(reproj_ref,reproj_sec,
                    hillshade_ref, hillshade_sec,
                    svf_ref, svf_sec,
                    colorbar_range)

## DEMs curvature

With `DemProcessing`, compute the curvature of the two DEMs, independently.  
For a DEM over a city, the curvature spotlights the buildings edges really efficiently. Comparing it to the reference curvature gives a good indication on the quality of the building restitution in a DEM. It also brings to light some artefacts which appeared during DEM generation: a tiling pattern can be observed. This is caused by the correlator used to generate the DEM. 

In [ ]:
dem_processing_object_ref_curvature = DemProcessing("ref-curvature")
ref_curvature = dem_processing_object_ref_curvature.process_dem(reproj_ref, reproj_sec)
dem_processing_object_sec_curvature = DemProcessing("sec-curvature")
sec_curvature = dem_processing_object_sec_curvature.process_dem(reproj_ref, reproj_sec)

Show the curvature of the two DEMs side-by-side

In [ ]:
plot_side_by_side(ref_curvature, 
                  sec_curvature,
                  "Reference DEM Curvature", 
                  "Second DEM Curvature",
                  -2,2,
                  -2,2)

## DEMs slope orientation histogram

Get the `slope_orientation_histogram`s for the two DEMs independently (it is a DEM statistics).  
The slope orientation histogram helps to analyze the main orientations of a DEM. It is useful to detect some artefacts, through some peaks that can be observed in the reference DEM in the main direction of the grid.

In [ ]:
%%capture
stats_dataset_ref2 = stats_processing_ref.compute_stats(metrics = ["slope-orientation-histogram"])
stats_dataset_sec2 = stats_processing_sec.compute_stats(metrics = ["slope-orientation-histogram"])
slope_orientation_histogram_ref = stats_dataset_ref2.get_classification_layer_metric(classification_layer = 'global', metric="slope-orientation-histogram", classif_class=0)
slope_orientation_histogram_sec = stats_dataset_sec2.get_classification_layer_metric(classification_layer = 'global', metric="slope-orientation-histogram", classif_class=0)

Plot the `slope_orientation_histogram`s on the same figure. 

In [ ]:
plot_two_slope_orientation_histogram(slope_orientation_histogram_ref[1], slope_orientation_histogram_ref[0],
                                     slope_orientation_histogram_sec[1], slope_orientation_histogram_sec[0])

# Comparing the two DEMs together

In this section, we compare the 2 DEMs together using different methods:
- the difference in altitude
- the slope-normalized altitude difference
- the angular difference

## Elevation difference

With `DemProcessing`, compute the difference in altitude.

In [ ]:
dem_processing_object_alti_diff = DemProcessing("alti-diff")
altitude_diff = dem_processing_object_alti_diff.process_dem(reproj_ref, reproj_sec)

With `DemProcessing`, compute the difference in altitude bewteen the two DEMs, and normalize it by the slope.  
The normalisation is very interesting as the elevation difference is intrinsically biased by the slope: important slopes tend to accentuate the elevation difference. Then, this method can reveal the areas in the DEM where the differences can actually be reduced as they would not directly result from the slope.

In [ ]:
dem_processing_object_alti_diff_slope_norm = DemProcessing("alti-diff-slope-norm")
altitude_diff_slope_norm = dem_processing_object_alti_diff_slope_norm.process_dem(reproj_ref, reproj_sec)

Show the difference in altitude, and the difference in altitude normalized by the slope side-by-side.

In [ ]:
plot_side_by_side(altitude_diff, 
                  altitude_diff_slope_norm,
                  "Elevation difference (ref-sec)", 
                  "Slope-normalized elevation difference (ref-sec)",
                  -4,4,
                  -1.5,1.5)

Create object from `StatsProcessing` with configuration and computed altitudes differences (classic and slope-normalized). 

In [ ]:
stats_processing_diff = StatsProcessing(cfg['statistics'], altitude_diff)
stats_dataset_diff = stats_processing_diff.compute_stats()

Get the pdf (Probability Density Function) and the cdf (Cumulative Density Function) metrics from these two methods and plot them.

In [ ]:
pdf_diff = stats_dataset_diff.get_classification_layer_metric(classification_layer = 'global', metric="pdf")
cdf_diff = stats_dataset_diff.get_classification_layer_metric(classification_layer = 'global', metric="cdf")

Select and calculate the scalar metrics requested in the configuration and store the results in a StatsDataset object.

In [ ]:
stats_metrics = stats_dataset_diff.get_classification_layer_metrics(classification_layer="global")
stats_metrics = stats_metrics[:-2]   # remove the 2 last metrics (nbpts and percent_valid_points) which are automatically selected
stats_metrics = stats_metrics[:-2]   # remove the cdf and pdf from the StatsDataset object as they are vector metrics and are displayed separately before

table_metrics = [["Metric", "Measured value (m)"]]
for metric in stats_metrics: 
    metric_value_diff = stats_dataset_diff.get_classification_layer_metric(classification_layer="global", metric=metric)
    table_metrics.append([metric, metric_value_diff[0]])

Plot the pdf and the cdf and display the scalar metrics for the elevation difference.  
Such metrics and statistics can help to assess the biases or the precision of a DEM.

In [ ]:
plot_cdf_pdf_side_by_side(pdf_diff, cdf_diff, "elevation difference")
display(HTML(tabulate.tabulate(table_metrics, tablefmt='html')))

Do the same for the slope-normalized elevation difference.

In [ ]:
stats_processing_diff_norm = StatsProcessing(cfg['statistics'], altitude_diff_slope_norm)
stats_dataset_diff_norm = stats_processing_diff_norm.compute_stats()

pdf_diff_norm = stats_dataset_diff_norm.get_classification_layer_metric(classification_layer = 'global', metric="pdf")
cdf_diff_norm = stats_dataset_diff_norm.get_classification_layer_metric(classification_layer = 'global', metric="cdf")

table_metrics_norm = [["Metric", "Measured value (m)"]]
for metric in stats_metrics: 
    metric_value = stats_dataset_diff_norm.get_classification_layer_metric(classification_layer="global", metric=metric)
    table_metrics_norm.append([metric, metric_value[0]])
    
plot_cdf_pdf_side_by_side(pdf_diff_norm, cdf_diff_norm, "elevation difference")
display(HTML(tabulate.tabulate(table_metrics_norm, tablefmt='html')))

## Angular difference

With `DemProcessing`, compute the angular difference between the two DEMs.  
The angular difference is useful to capture shape divergences and distorsions. It can also be helpful to analyze and understand some local details (such as the vegetation recosntruction).

In [ ]:
dem_processing_object_angular_diff = DemProcessing("angular-diff")
angular_diff = dem_processing_object_angular_diff.process_dem(reproj_ref, reproj_sec)

Get the metrics for the angular difference.

In [ ]:
stats_processing_ang = StatsProcessing(cfg['statistics'], angular_diff)
stats_dataset_ang = stats_processing_ang.compute_stats()

pdf_diff_ang = stats_dataset_ang.get_classification_layer_metric(classification_layer = 'global', metric="pdf")
cdf_diff_ang = stats_dataset_ang.get_classification_layer_metric(classification_layer = 'global', metric="cdf")

table_metrics_ang = [["Metric", "Measured value (rad)"]]
for metric in stats_metrics: 
    metric_value = stats_dataset_ang.get_classification_layer_metric(classification_layer="global", metric=metric)
    table_metrics_ang.append([metric, metric_value[0]])

Plot the angular difference with its associated pdf and cdf. It is possible to set the minimum and maximum values for the angular difference plot.

In [ ]:
vmin, vmax = 0, 1.6
plot_angular_diff(angular_diff,pdf_diff_ang,cdf_diff_ang,vmin,vmax)
display(HTML(tabulate.tabulate(table_metrics_ang, tablefmt='html')))